# Introduction

Real world datasets have surprises in them such as missing data, outliers, and data entry errors. Code you create to analyze this data has its own set of surprises such as logical errors, syntax errors and conceptual errors. In the interest of progress it can be helpful to reduce variables to just your code before introducing additional complexity. In this example we will generate a synthetic dataset and form a base regression model for the task.

Based on the approach described in https://d2l.ai/chapter_linear-regression/synthetic-regression-data.html

# Generating Data

We'll generate data from a pure linear function and then pollute the data with noise.

In [1]:
import torch
import random

# y = Xw + b + noise
class SyntheticRegressionData():
    def __init__(self, w, b, noise=0.1, training_count = 1000, validation_count = 1000, batch_size = 32):
        self.w = w
        self.b = b
        self.batch_size = batch_size
        self.training_count = training_count
        self.validation_count = validation_count
        self.observation_count = training_count + validation_count
        self.X = torch.randn(self.observation_count, len(w))
        noise = torch.randn(self.observation_count, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

    def get_tensorloader(self, tensors, train, indices=slice(0, None)):
        tensors = tuple(a[indices] for a in tensors)
        dataset = torch.utils.data.TensorDataset(*tensors)
        return torch.utils.data.DataLoader(dataset, self.batch_size, shuffle=train)
    
    def get_dataloader(self, train):
        if train:
            # Training data is in the front of the dataset
            i = slice(0, self.training_count)
        else:
            # Validation data is at the end of the dataset
            i = slice(self.training_count, None)
        return self.get_tensorloader((self.X, self.y), train, i)
    
    def training_dataloader(self):
        return self.get_dataloader(train=True)

    def validation_dataloader(self):
        return self.get_dataloader(train=False)
        

In [2]:
# Try our new class
data = SyntheticRegressionData(w=torch.tensor([5, -2.1]), b=3.1)
print(f"Features: {data.X[0]}")
print(f"Label: {data.y[0]}")

Features: tensor([ 1.6024, -0.4316])
Label: tensor([11.9688])


In [3]:
# Try out the training dataloader
X, y = next(iter(data.training_dataloader()))
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: torch.Size([32, 2])
y shape: torch.Size([32, 1])


# Define the Optimizer (From Scratch)

As descibed in the article d2l.ai, we'll use mini-batch stochastic gradient descent for our optimizer

In [4]:
class SGD():
    def __init__(self, params, learning_rate):
        self.params = params
        self.learning_rate = learning_rate

    def step(self):
        for param in self.params:
            param -= self.learning_rate * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

# Define the Model (From Scratch)

We'll use only primitives for the first model and then we'll reimplement using more features in the Torch framework

In [5]:
from torch import nn

class LinearRegressionV1(torch.nn.Module):
    def __init__(self, input_count, learning_rate, sigma=0.01):
        super().__init__()
        self.learning_rate = learning_rate
        # Draw data from the normal distribution centered at 0
        # with standard deviation sigma
        self.w = torch.normal(0, sigma, (input_count, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def forward(self, X):
        # y = Xw + b
        return torch.matmul(X, self.w) + self.b

    def loss(self, y_predicted, y):
        # We use the squared loss function
        l = ((y_predicted - y) ** 2) / 2
        return l.mean()

    def training_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        return l

    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        return l

    def configure_optimizers(self):
        return SGD([self.w, self.b], self.learning_rate)
        

# Training

In each epoch, we process an iteration of a random batch of data. We continue processing iterations until all data has been seen.

For each iteration we do the following
 1. Grab a batch of training samples
 2. Compute the loss
 3. Compute the gradient on where to move next
 4. Update the model parameters

In [6]:
class MiniBatchTrainer():
    def __init__(self, max_epochs):
        self.max_epochs = max_epochs

    def prepare_data(self, data):
        self.training_dataloader = data.training_dataloader()
        self.validation_dataloader = data.validation_dataloader()
        self.training_batch_count = len(self.training_dataloader)
        self.validation_batch_count = len(self.validation_dataloader)

    def prepare_model(self, data):
        model.trainer = self
        self.model = model

    def fit_epoch(self, _callback = None):
        self.model.train()
        for batch in self.training_dataloader:
            loss = self.model.training_step(batch)
            if _callback:
                _callback("training loss", loss)
            self.optimizer.zero_grad()
            with torch.no_grad():
                loss.backward()
                self.optimizer.step()
                
            self.training_batch_index += 1

        self.model.eval()
        for batch in self.validation_dataloader:
            with torch.no_grad():
                loss = self.model.validation_step(batch)
                if _callback:
                    _callback("validation loss", loss)
            self.validation_batch_index +=1
    
    def fit(self, model, data, _callback = None):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optimizer = model.configure_optimizers()
        self.epoch = 0
        self.training_batch_index = 0
        self.validation_batch_index = 0
        for self.epoch in range(self.max_epochs):
            _callback("epoch", self.epoch)
            self.fit_epoch(_callback)

# All Together

We'll bring it all together to declare the model, generate synthetic data and train the model.

In [7]:
def status_callback(topic, status):
    print(f"{topic}: {status}")

model = LinearRegressionV1(2, learning_rate=0.03)
data = SyntheticRegressionData(w=torch.tensor([5, -2.1]), b=3.1, training_count=256, validation_count=64)
trainer = MiniBatchTrainer(max_epochs=10)
trainer.fit(model, data, status_callback)

epoch: 0
training loss: 22.99555015563965
training loss: 17.691625595092773
training loss: 16.753314971923828
training loss: 21.867164611816406
training loss: 18.695274353027344
training loss: 9.408775329589844
training loss: 10.600300788879395
training loss: 13.083674430847168
validation loss: 8.337923049926758
validation loss: 11.775568008422852
epoch: 1
training loss: 11.873462677001953
training loss: 9.059558868408203
training loss: 12.04800033569336
training loss: 8.084768295288086
training loss: 10.337409019470215
training loss: 9.060818672180176
training loss: 8.499825477600098
training loss: 9.792357444763184
validation loss: 4.993284225463867
validation loss: 7.082032680511475
epoch: 2
training loss: 6.207742214202881
training loss: 6.747061729431152
training loss: 7.571684837341309
training loss: 7.4557085037231445
training loss: 5.543828964233398
training loss: 4.577333450317383
training loss: 6.803302764892578
training loss: 2.4466588497161865
validation loss: 2.98778986930

In [8]:
with torch.no_grad():
    print(f"Error in estimating w: {data.w - model.w.reshape(data.w.shape)}")
    print(f"Error is estimating b: {data.b - model.b}")

Error in estimating w: tensor([ 0.3824, -0.1682])
Error is estimating b: tensor([0.2568])
